In [1]:
import pandas as pd 
import pandas_profiling as pp 
aguadataframe = pd.read_csv('Calidad_de_agua_2022.csv',delimiter = ';')
aguadataframe
aguadataframe.info()

c:\Users\ramir\Fundamentos\Practicoespecial\PracticoEspecialGrupo14\practicoEspecial\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   orden                    168 non-null    int64 
 1   sitios                   168 non-null    object
 2   codigo                   168 non-null    object
 3   fecha                    168 non-null    object
 4   año                      168 non-null    object
 5   campaña                  168 non-null    object
 6   tem_agua                 168 non-null    object
 7   tem_aire                 166 non-null    object
 8   od                       168 non-null    object
 9   ph                       168 non-null    object
 10  olores                   168 non-null    object
 11  color                    168 non-null    object
 12  espumas                  168 non-null    object
 13  mat_susp                 168 non-null    object
 14  colif_fecales_ufc_100ml  167 non-null    o

C:\Users\ramir\AppData\Local\Temp\ipykernel_7804\1932533977.py:2: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling as pp


viendo la informacion del dataset, se puede ver que hay algunas columnas que contienen valores nulos (informacion para un posterior analisis) y ademas que todas las columnas son de tipo objet lo cual nos da un indicio de que algunas columnas vamos a tener que transformarlas y cambiarlas de tipo. 

In [2]:
aguadataframe = aguadataframe.drop(columns='orden')

tomamos la decision de eliminar la columna orden debido a que no aportaba ningun tipo de informacion para el analisis de los datos. 

In [3]:
aguadataframe.columns
aguadataframe.describe()

,sitios,codigo,fecha,año,campaña,tem_agua,tem_aire,od,ph,olores,...,dbo_mg_l,dqo_mg_l,turbiedad_ntu,hidr_deriv_petr_ug_l,cr_total_mg_l,cd_total_mg_l,clorofila_a_ug_l,microcistina_ug_l,ica,calidad_de_agua
count,168,168,168,168,168,168,166,168,168,168,...,167,168,168,168,168,168,167,168,155,154
unique,42,42,7,3,6,93,32,130,116,6,...,55,51,63,6,27,4,70,15,39,3
top,Canal Villanueva y Río Luján,TI001,23/2/2022,2022,Verano,no se midió,14,no se midió,no se midió,Ausencia,...,no se midió,<30,no se midió,<0.10,<0.005,<0.001,<10,<0.15,40,Extremadamente deteriorada
freq,4,4,42,163,42,19,25,24,23,120,...,53,51,10,113,113,147,74,76,13,92


como vemos cuando hacemos un describe nos muestra pocos datos y omite los estadisticos como la media, los cuartiles, etc. Esto es debido a que todas las variables son de tipo object asique lo que habria que hacer es convertirlas a algun otro tipo para poder analizarlas. 

In [4]:
aguadataframe1= aguadataframe.replace("no se midio",pd.NA)
aguadataframe1 = aguadataframe.replace({"<0.10": 0.10, "<0.005": 0.005, "<0.001": 0.001, "<0.15": 0.15, "<30": 30})
columnas_numericas=['tem_agua', 'tem_aire', 'od', 'ph', 'dbo_mg_l', 'dqo_mg_l', 
                      'turbiedad_ntu', 'hidr_deriv_petr_ug_l', 'cr_total_mg_l', 
                      'cd_total_mg_l', 'clorofila_a_ug_l', 'microcistina_ug_l', 'ica']
aguadataframe1[columnas_numericas]=aguadataframe[columnas_numericas].apply(pd.to_numeric, errors='coerce')
print(aguadataframe1.describe())


         tem_agua    tem_aire          od          ph   dbo_mg_l    dqo_mg_l  \
count  145.000000  144.000000  132.000000  112.000000  88.000000   96.000000   
mean    17.823655   15.793056    6.747348    7.570000   7.065909   51.302083   
std      4.848688    5.124351    2.835764    0.770363   5.671544   26.944043   
min      6.000000    4.000000    0.360000    5.000000   1.900000   29.000000   
25%     14.600000   13.000000    5.065000    7.075000   3.500000   29.000000   
50%     17.900000   14.000000    6.650000    7.485000   5.550000   41.500000   
75%     20.400000   19.000000    8.580000    8.002500   9.325000   64.250000   
max     27.400000   27.000000   17.610000   10.020000  42.000000  180.000000   

       turbiedad_ntu  hidr_deriv_petr_ug_l  cr_total_mg_l  cd_total_mg_l  \
count     149.000000                   0.0      37.000000            0.0   
mean       35.240268                   NaN       2.113768            NaN   
std        24.106259                   NaN       3.

ahora que transformamos los valores se puede ver un mejor analisis del metodo describe, por ejemplo se puede ver la media de la temperatura del agua que es de 17.82, el desvio, los valores maximos y minimos y los cuartiles.  Para las columnas que tienen valores que dicen "ausencia", "presencia", las vamos a convertir en booleanas para por ejemplo analizar la frecuencia o analizar algun tipo de correlacion con alguna otra variable.

In [18]:
boolean_columns = ['espumas', 'olores', 'color', 'mat_susp']
for col in boolean_columns:
    aguadataframe1[col] = aguadataframe1[col].replace({'Presencia': True, 'Ausencia': False})
for col in boolean_columns:
    aguadataframe1[col] = aguadataframe1[col].astype(bool)
aguadataframe1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sitios                   168 non-null    object 
 1   codigo                   168 non-null    object 
 2   fecha                    168 non-null    object 
 3   año                      168 non-null    object 
 4   campaña                  168 non-null    object 
 5   tem_agua                 145 non-null    float64
 6   tem_aire                 144 non-null    float64
 7   od                       132 non-null    float64
 8   ph                       112 non-null    float64
 9   olores                   168 non-null    bool   
 10  color                    168 non-null    bool   
 11  espumas                  168 non-null    bool   
 12  mat_susp                 168 non-null    bool   
 13  colif_fecales_ufc_100ml  167 non-null    object 
 14  escher_coli_ufc_100ml    1